In [121]:
import prepare_models
import os
import cv2 as cv
import numpy as np
from matplotlib.image import imread
import scipy.misc
from PIL import Image  
import glob
from matplotlib.image import imread
import imutils

In [2]:
models = prepare_models.Models()
extraction_predictor = models.preapre_extraction_model()
segmentation_predictor = models.preapre_segmentation_model()

Model initialization


In [143]:
labelme_start = '''{
"version": "4.5.10",
"flags": {},
"shapes": [
'''
def labelme_end(filename,h,w):
    labelme_end = '''],
"imagePath": "{}",
"imageData": null,
"imageHeight": {},
"imageWidth": {}
'''.format(filename,h,w)
    return labelme_end

def add_points(label,contours):
    # Approximate the contour
    eps = 0.009 # Approximation parameter: higher ==> less points
    contours = imutils.grab_contours(contours)
    contours = max(contours, key=cv.contourArea)
    peri = cv.arcLength(contours, True)
    approx = cv.approxPolyDP(contours, eps * peri, True)
    cont = np.vstack(approx).squeeze().tolist()
    content = '''
"label": "{}",
"points": {},
"group_id": null,
"shape_type": "polygon",
"flags": {}
'''.format(label,cont,'{}')
    return str(content)

failures_dictionary = {
    0:"wykruszenie",
    1:"narost",
    2:"stepienie",
    3:"zatarcie"   
}

In [146]:
pth = r'D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\\'

for imageName in glob.glob(pth + '*png'):
    print(imageName)
    im = cv.imread(imageName)
    outputs = segmentation_predictor(im) # Make prediction
    name = imageName.split('\\')[-1]
    img = Image.fromarray(im)
    base_name = name.split('.')[0]
    # Get mask and label from the prediction
    pred_masks = outputs["instances"].to("cpu").pred_masks.numpy()
    pred_classes = outputs["instances"].to("cpu").pred_classes.numpy()
    num_instances = pred_masks.shape[0] 
    pred_masks = np.moveaxis(pred_masks, 0, -1)
    pred_masks_instance = []
    output = np.zeros_like(im)

    labelme_json = labelme_start # Create begening of the labelme json file
    for i in range(num_instances): # Iterate over predicted defects instances and add it to the labelme json file
        labelme_json += '{' # Open container for new instance    
        
        # Get binary mask from inference, convert it into opencv format and find contours
        pred_masks_instance.append(pred_masks[:, :, i:(i+1)]) 
        output = np.where(pred_masks_instance[i] == True, 255, output)
        im = Image.fromarray(output)
        output = np.zeros_like(im)
        pred_masks_instances = []
        im = np.array(im) 
        im = cv.cvtColor(im, cv.COLOR_BGR2GRAY )   
        contours = cv.findContours(im, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

        # Add contours to the labelme json file
        labelme_json += add_points(failures_dictionary[pred_classes[i]],contours)
        
        # Check if it is last instance and close container
        if(i+1 == num_instances):labelme_json += '}\n' 
        else: labelme_json += '},\n'
  

    labelme_json += labelme_end(base_name + '.png',img.size[1],img.size[0]) # Add some additional information
    labelme_json += '}' # End of the labelme json file
    out_txt_name = pth + base_name  + '.json'
    # Save labelme json file
    file = open(out_txt_name, 'w')
    file.write(str(labelme_json))
    file.close() 

print("Finished")   

D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_030.png
D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_036.png
D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_043.png
D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_049.png
D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_055.png
D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_061.png
D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_067.png
D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_073.png
D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_080.png
D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_086.png
D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_092.png
D:\Konrad\TCM_scan\traning_segmentation\data\auto_anot\20210621_092043_056_098.png
D:\K

'stepienie'

In [127]:
if (inst_ids[i]=='1' and (1 in draw_pick)): # blue - 'stępienie' 
                g = cv.bitwise_and(g,cv.split(mask)[0])
                r = cv.bitwise_and(r,cv.split(mask)[0])
            elif(inst_ids[i]=='2'and (2 in draw_pick)): # red - 'narost'
                g = cv.bitwise_and(g,cv.split(mask)[0])
                b = cv.bitwise_and(b,cv.split(mask)[0])
            elif(inst_ids[i]=='3'and (3 in draw_pick)): # green - 'zatarcie'
                r = cv.bitwise_and(r,cv.split(mask)[0])
                b = cv.bitwise_and(b,cv.split(mask)[0])
            elif(inst_ids[i]=='4'and (4 in draw_pick)): # yellow - 'wykruszenie'

PIL.Image.Image